In [6]:
from __future__ import print_function
from  torch import nn, optim, cuda
from torch.utils import data
from torchvision import datasets, transforms
import torch.nn.functional as F
import time


In [7]:
#트레이닝 설정
batch_size = 64
device = 'cuda' if cuda.is_available() else 'cpu'
print(f'Training MNIST model on {device} \n{"=" * 44}')



Training MNIST model on cuda 


In [8]:
#MNIST 데이터 셋
train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)
test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())




In [9]:
# Data Loader
train_loader = data.DataLoader(dataset= train_dataset,
                               batch_size=batch_size,
                               shuffle=True)
test_loader = data.DataLoader(dataset=test_dataset,
                              batch_size=batch_size,
                              shuffle=True)


In [10]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(784, 520)
        self.l2 = nn.Linear(520, 320)
        self.l3 = nn.Linear(320, 240)
        self.l4 = nn.Linear(240, 120)
        self.l5 = nn.Linear(120, 10)

    def forward(self, x):
        x = x.view(-1, 784) # Flatten the data (n, 1, 28, 28) -> (n, 784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)



In [11]:
model = Net()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


In [12]:
def train(epoch):
    model.train()
    for batch_idx,(data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} | Barch Status: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset), 100. * batch_idx / len(train_loader), loss.item()))



In [13]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(f'==============================\nTest set: Average loss : {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)}'
          f'({100. * correct / len(test_loader.dataset):.0f}%)')



In [14]:
if __name__ == '__main__':
    since = time.time()
    for epoch in range(1, 10):
        epoch_start = time.time()
        train(epoch)
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Training Time: {m:.0f}m {s:.0f}s')
        test()
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Testing time: {m:.0f}m {s:.0f}s')

    m, s = divmod(time.time() - since, 60)
    print(f'Total Time: {m:.0f}m {s:.0f}s\nModel was trained on {device}!')



Train Epoch: 1 | Barch Status: 0/60000 (0%) | Loss: 2.307881
Train Epoch: 1 | Barch Status: 640/60000 (1%) | Loss: 2.299308
Train Epoch: 1 | Barch Status: 1280/60000 (2%) | Loss: 2.308011
Train Epoch: 1 | Barch Status: 1920/60000 (3%) | Loss: 2.286199
Train Epoch: 1 | Barch Status: 2560/60000 (4%) | Loss: 2.302287
Train Epoch: 1 | Barch Status: 3200/60000 (5%) | Loss: 2.300444
Train Epoch: 1 | Barch Status: 3840/60000 (6%) | Loss: 2.306627
Train Epoch: 1 | Barch Status: 4480/60000 (7%) | Loss: 2.299743
Train Epoch: 1 | Barch Status: 5120/60000 (9%) | Loss: 2.306314
Train Epoch: 1 | Barch Status: 5760/60000 (10%) | Loss: 2.309899
Train Epoch: 1 | Barch Status: 6400/60000 (11%) | Loss: 2.296050
Train Epoch: 1 | Barch Status: 7040/60000 (12%) | Loss: 2.304537
Train Epoch: 1 | Barch Status: 7680/60000 (13%) | Loss: 2.304069
Train Epoch: 1 | Barch Status: 8320/60000 (14%) | Loss: 2.302695
Train Epoch: 1 | Barch Status: 8960/60000 (15%) | Loss: 2.300954
Train Epoch: 1 | Barch Status: 9600/60